<a href="https://colab.research.google.com/github/VectorJamo/Deep-Learning/blob/main/Convolutional_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms # Use out of the box data such as the MNIST images
from torchvision.utils import make_grid

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Convert MNIST images into a 4D Tensor (no. of images, height, width and the color channels)
transform = transforms.ToTensor()

In [3]:
# Training data
training_data = datasets.MNIST(root='/cnn_data', train=True, download=True, transform=transform)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9.91M/9.91M [00:10<00:00, 904kB/s] 


Extracting /cnn_data/MNIST/raw/train-images-idx3-ubyte.gz to /cnn_data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28.9k/28.9k [00:00<00:00, 134kB/s]


Extracting /cnn_data/MNIST/raw/train-labels-idx1-ubyte.gz to /cnn_data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1.65M/1.65M [00:01<00:00, 1.26MB/s]


Extracting /cnn_data/MNIST/raw/t10k-images-idx3-ubyte.gz to /cnn_data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4.54k/4.54k [00:00<00:00, 10.4MB/s]

Extracting /cnn_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /cnn_data/MNIST/raw



In [4]:
# Testing data
testing_data = datasets.MNIST(root='/cnn_data', train=False, download=True, transform=transform)

In [5]:
training_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: /cnn_data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [6]:
testing_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: /cnn_data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [7]:
# Create small batches of images
batch_size = 10
train_loader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(testing_data, batch_size=batch_size, shuffle=False)

In [8]:
# Define our CNN Model
# Describle the convolutional layer (2 convolutional layers)
conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=3, stride=1) # no. of channels(color data) in the input image (1), use 6 distinct filters each of size 3x3, stride is 1
conv2 = nn.Conv2d(6, 16, 3, 1) # no. of channels = 6, since the previous convolution layer outputs 6 volumes given by 6 distinct filters, use 16 distinct filters each of size 3x3, stride is 1

In [9]:
# Grab 1 MNIST image
for i, (X_train, y_train) in enumerate(training_data):
  break

In [10]:
X_train.shape # 1 image of size 28x28

torch.Size([1, 28, 28])

In [12]:
y_train # Value represented by that image

5

In [15]:
x = X_train.view(1, 1, 28, 28) # Reshape the X_train tensor in a new shape. 1 batch(1 image), 1 channel(black and white) of size 28x28

In [17]:
# Perform our first convolution
x = F.relu(conv1(x)) # Use relu as the activation function

In [20]:
x.shape # output of that 1 image after applying each of the 6 filters, which is 6 distinct 26x26 images.

torch.Size([1, 6, 26, 26])

In [21]:
# Pass through the pooling layer
x = F.max_pool2d(x, kernel_size=2, stride=2) # "Compress" the output of each of the 6 filters by applying a max pooling method.

In [22]:
x.shape

torch.Size([1, 6, 13, 13])

In [23]:
# Do the second convolution layer
x = F.relu(conv2(x))

In [24]:
x.shape

torch.Size([1, 16, 11, 11])

In [25]:
# Pooling layer
x = F.max_pool2d(x, kernel_size=2, stride=2)

In [26]:
x.shape

torch.Size([1, 16, 5, 5])

In [27]:
# Condense everything we did above into a Model class
class ConvolutionalNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=3, stride=1)
    self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=3, stride=1)

    # Fully connected layer
    self.fc1 = nn.Linear(in_features=16*5*5, out_features=120) # Input features are the each individual data obtained after convolution and pooling.
                                                               # In our case, the final result after pooling (for 1 image) was 16 "compressed" images of size 5x5
                                                               # Output features is 120 which is open to tweaking.

    self.fc2 = nn.Linear(in_features=120, out_features=84)     # Output features is 84 which is open to tweaking.
    self.fc3 = nn.Linear(in_features=84, out_features=10)      # Output features for the final output layer is 10 since we have 10 digits to recognize.

  def forward(self, X):
    # Define how the input data will flow through the CNN

    # First convolution
    X = F.relu(self.conv1(X))
    # First pooling
    X = F.max_pool2d(X, kernel_size=2, stride=2)
    # Second convolution
    X = F.relu(self.conv2(X))
    # Second pooling
    X = F.max_pool2d(X, kernel_size=2, stride=2)

    # Flatten this multidimension tensor into a 1D tensor so that we can pass it into the fully connected neural network layers
    X = X.view(-1, 16*5*5) # -1 so that we can vary the batch size.

    # Pass through the fully connected layers
    X =  F.relu(self.fc1(X))
    X =  F.relu(self.fc2(X))
    X = self.fc3(X)

    return F.log_softmax(X, dim=1)

In [30]:
# Create an instance of the model
torch.manual_seed(41)
model = ConvolutionalNetwork()
model

ConvolutionalNetwork(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [31]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [35]:
import time
start_time = time.time()

# Create variables to track things
epochs = 5
train_losses = []
test_losses = []
train_correct = []
test_correct = []

# For loop of epochs
for i in range(epochs):
  train_corr = 0
  test_corr = 0

  # Train
  for b, (X_train, y_train) in enumerate(train_loader):
    b += 1 # Start our batches at 1

    # Apply our model
    y_pred = model(X_train) # Run the model
    loss = criterion(y_pred, y_train) # Calculate the loss. Compare the prediction to the actual label.

    predicted = torch.max(y_pred.data, 1)[1] # Returns the maximum values and their indices. Get the index of the class with the highest probability score.
    batch_corr = (predicted == y_train).sum() # Calculate the number of correct predictions
    train_corr += batch_corr

    # Update our parameters
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print out some results/progress
    if b % 600 == 0:
      print(f'Epoch: {i} Batch: {b} Loss: {loss.item()}')

  train_losses.append(loss)
  train_correct.append(train_corr)

  # Test
  with torch.no_grad():
    for b, (X_test, y_test) in enumerate(test_loader):
      y_val = model(X_test)
      predicted = torch.max(y_val.data, 1)[1]
      test_corr += (predicted == y_test).sum()

    loss = criterion(y_val, y_test)
    test_losses.append(loss)
    test_correct.append(test_corr)

current_time = time.time()
time_taken = current_time - start_time
print(f'Training took: {time_taken/60.0} minutes!')

Epoch: 0 Batch: 600 Loss: 0.2649766802787781
Epoch: 0 Batch: 1200 Loss: 0.017863785848021507
Epoch: 0 Batch: 1800 Loss: 0.006464975420385599
Epoch: 0 Batch: 2400 Loss: 0.036078017204999924
Epoch: 0 Batch: 3000 Loss: 0.10666809231042862
Epoch: 0 Batch: 3600 Loss: 0.020617660135030746
Epoch: 0 Batch: 4200 Loss: 0.01998436637222767
Epoch: 0 Batch: 4800 Loss: 0.00944086629897356
Epoch: 0 Batch: 5400 Loss: 0.015996595844626427
Epoch: 0 Batch: 6000 Loss: 0.41673699021339417
Epoch: 1 Batch: 600 Loss: 0.04192870110273361
Epoch: 1 Batch: 1200 Loss: 0.43111753463745117
Epoch: 1 Batch: 1800 Loss: 0.005686578806489706
Epoch: 1 Batch: 2400 Loss: 0.005317699629813433
Epoch: 1 Batch: 3000 Loss: 0.003176052588969469
Epoch: 1 Batch: 3600 Loss: 0.0012339743552729487
Epoch: 1 Batch: 4200 Loss: 0.01875397562980652
Epoch: 1 Batch: 4800 Loss: 0.0004435688315425068
Epoch: 1 Batch: 5400 Loss: 0.016413195058703423
Epoch: 1 Batch: 6000 Loss: 0.030590971931815147
Epoch: 2 Batch: 600 Loss: 0.019589880481362343
Ep